### Using Openzim Youtube2zim program
* This program is written by Kiwix and has a variety of methods for specifying which videos to include in a ZIM file. Unfortunately, a flat file of youtube ID's is not one of them. But one of the input specifications that is available is to specify a "Youtube Playlist".
* Getting permission to write a playlist from a python program is a task.
    1. The written playlist can only be written to by a user who has a google account, and who also owns a Youtube Channel.
    2. with proper credentials, the owner can create a playlist of up to 5000 videos in her own channel.
    3. Writing to youtube requires an oauth2 credential. The steps I found necessary:
        1. Go to "console.developer.google.com" to create a project.
        2. Within that project create a key (for reading public data) and a oauth2 credential for writing.
        3. The credential needs to specify a callback where google can return a software key which includes the permission that the owner of a Youtube channel gives to the python program. The callback I used was a flask program at iiab-factory/content/kiwix/ted-tools/flask_oauth_youtuve.py. And the URL created by this flask program needs to be specified to google during the process of creating the oauth2 credential as "http://localhost:8088/oauth2callback".
        4. The flask program writes a file in the current directory "zim_playbook_oauth_credentials".

In [4]:
# -*- coding: utf-8 -*-
import os,sys
import json

PREFIX = os.environ.get('ZIM_PREFIX','/ext/zims')
# Declare a short project name (ZIM files are often long strings
#PROJECT_NAME = 'ted-kiwix'
PROJECT_NAME = 'youtube'
PREFIX = os.environ.get('ZIM_PREFIX','/ext/zims')
TARGET_SIZE =10000000000  #10GB
# Input the full path of the downloaded ZIM file
ZIM_PATH = '%s/%s/working/wanted_list.csv'%(PREFIX,PROJECT_NAME,) 
# The rest of the paths are computed and represent the standard layout
# Jupyter sets a working director as part of it's setup. We need it's value
HOME = os.environ['HOME']
WORKING_DIR = PREFIX + '/' + PROJECT_NAME + '/working'
PROJECT_DIR = PREFIX + '/' + PROJECT_NAME + '/tree'
OUTPUT_DIR = PREFIX + '/' + PROJECT_NAME + '/output_tree'
SOURCE_DIR = PREFIX + '/' + PROJECT_NAME + '/zim-src'
dir_list = ['output_tree','tree','working/video_json','zim-src'] 
for f in dir_list:
    if not os.path.isdir(PREFIX + '/' + PROJECT_NAME +'/' + f): os.makedirs(PREFIX + '/' + PROJECT_NAME +'/' + f) 
# abort if the input file cannot be found
if not os.path.exists(ZIM_PATH):
    print('%s path not found. Quitting. . .'%ZIM_PATH)
    exit


In [12]:

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
# The User validated google credentials are in her home directory
home = os.environ['HOME']
with open("%s/zim_playbook_oauth_credentials"%home,'r') as fp:
    credentials = json.loads(fp.read())
    
# Load credentials from file
credentials = google.oauth2.credentials.Credentials(**credentials)
 
youtube = googleapiclient.discovery.build(
    'youtube', 'v3', credentials=credentials)

channel = youtube.channels().list(mine=True, part='snippet').execute()

# check that the authentication worked
#print(json.dumps(channel,indent=2))

{
  "kind": "youtube#channelListResponse",
  "etag": "tRVB409M78PdigH7dSlPazTvgAg",
  "pageInfo": {
    "totalResults": 1,
    "resultsPerPage": 5
  },
  "items": [
    {
      "kind": "youtube#channel",
      "etag": "59ppxg9_8rXxxZfSlBdYPXm1Xeg",
      "id": "UCljl2cmQMgzlTkeJqJier7g",
      "snippet": {
        "title": "George Hunt",
        "description": "",
        "publishedAt": "2011-10-06T18:43:32Z",
        "thumbnails": {
          "default": {
            "url": "https://yt3.ggpht.com/ytc/AAUvwngpZkDZ2rzp1hzCGEUH6C5SnKXbNPK6galHUYLb1g=s88-c-k-c0x00ffffff-no-rj",
            "width": 88,
            "height": 88
          },
          "medium": {
            "url": "https://yt3.ggpht.com/ytc/AAUvwngpZkDZ2rzp1hzCGEUH6C5SnKXbNPK6galHUYLb1g=s240-c-k-c0x00ffffff-no-rj",
            "width": 240,
            "height": 240
          },
          "high": {
            "url": "https://yt3.ggpht.com/ytc/AAUvwngpZkDZ2rzp1hzCGEUH6C5SnKXbNPK6galHUYLb1g=s800-c-k-c0x00ffffff-no-rj",
    